In [ ]:
!pip install pyspark

# Perguntas dessa tarefa

## 1.

- Inicialmente iremos preparar o ambiente, definindo o diretório onde nosso código será desenvolvido. Para este diretório iremos copiar o arquivo nomes_aleatorios.txt.

- Após, em nosso script Python, devemos importar as bibliotecas necessárias:

from pyspark.sql import SparkSession

from pyspark import SparkContext, SQLContext

- Aplicando as bibliotecas do Spark, podemos definir a Spark Session e sobre ela definir o Context para habilitar o módulo SQL

spark = SparkSession \

                .builder \

                .master("local[*]")\

                .appName("Exercicio Intro") \

                .getOrCreate()

- Nesta etapa, é a leitura dos arquivos de filmes e series CSV e JSON

In [66]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext

spark = SparkSession \
    .builder \
    .master("local[*]")\
    .appName("ELT TRUSTED") \
    .getOrCreate()

path1 = f"/mnt/wsl/PHYSICALDRIVE2/Projects/Compasso/Azimute/Sprint_7/Desafio01_ETL/assets/"
df_movies_IMDB = spark.read.option("delimiter", "|").csv(path1 + 'movies.csv', header=True)
df_series_IMDB = spark.read.option("delimiter", "|").csv(path1 + 'series.csv', header=True)

path2 = f"/mnt/wsl/PHYSICALDRIVE2/Projects/Compasso/Azimute/Sprint_8/Desafio02_ETL/Tarefa_02/assets/"
df_movies_TMDB = spark.read.json(path2 + 'movies.json') 
df_series_TMDB = spark.read.json(path2 + 'series.json')

df_movies_IMDB.show(1)
df_movies_TMDB.show(1)

df_series_IMDB.show(1)
df_series_TMDB.show(1)

+---------+--------------+--------------+-------------+------------+-------+---------+-----------+-------------+--------------------+---------------+-------------+--------------+---------+---------------------+
|       id|tituloPincipal|tituloOriginal|anoLancamento|tempoMinutos| genero|notaMedia|numeroVotos|generoArtista|          personagem|    nomeArtista|anoNascimento|anoFalecimento|profissao|titulosMaisConhecidos|
+---------+--------------+--------------+-------------+------------+-------+---------+-----------+-------------+--------------------+---------------+-------------+--------------+---------+---------------------+
|tt0000009|    Miss Jerry|    Miss Jerry|         1894|          45|Romance|      5.3|        200|      actress|Miss Geraldine Ho...|Blanche Bayliss|         1878|          1951|  actress|            tt0000009|
+---------+--------------+--------------+-------------+------------+-------+---------+-----------+-------------+--------------------+---------------+-------

## 2. 
 Filtro os Filmes do BD Local com lista de filmes do TMDB

In [58]:
from pyspark.sql.functions import col, substring, when, size

# Converter a coluna 'title' do DataFrame JSON para um array de palavras e extrair o ano de lançamento
df_filmes_titulos_datas = df_movies_TMDB.select(col('title'), substring(col('release_date'), 1, 4).alias('releaseDate'))

# Definir o tamanho mínimo das palavras em comum
tamanho_minimo = 4

# Realizar a filtragem dos títulos
df_IMDB_filtrado_filmes = df_movies_IMDB.join(df_filmes_titulos_datas,
                                   (df_movies_IMDB['tituloPincipal'].contains(df_filmes_titulos_datas['title'])) &
                                   (df_movies_IMDB['anoLancamento'] == df_filmes_titulos_datas['releaseDate']),
                                   'inner')

# Remover as colunas 'title' e 'releaseDate' do DataFrame
df_IMDB_filtrado_filmes = df_IMDB_filtrado_filmes.drop('title', 'releaseDate')

# Renomear a coluna 'tituloPincipal' para 'tituloPrincipal' no DataFrame df_IMDB_filtrado_filmes
df_IMDB_filtrado_filmes = df_IMDB_filtrado_filmes.withColumnRenamed("tituloPincipal", "tituloPrincipal")

df_IMDB_filtrado_filmes = df_IMDB_filtrado_filmes.distinct()

# Trata os valores "NA" na coluna "anoFalecimento"
df_IMDB_Filmes_resultado = df_IMDB_filtrado_filmes.withColumn("anoFalecimento", when(col("anoFalecimento") == "\\N", None).otherwise(col("anoFalecimento")))

# Exibir resultado
df_IMDB_Filmes_resultado.show(10)
print("Número de linhas antes IMDB: ", df_movies_IMDB.count())
print("Número de linhas depois IMDB: ", df_IMDB_Filmes_resultado.count())
print("Número de linhas TMDB: ", df_movies_TMDB.count())

+---------+--------------------+--------------------+-------------+------------+--------------------+---------+-----------+-------------+----------------+--------------------+-------------+--------------+--------------------+---------------------+
|       id|     tituloPrincipal|      tituloOriginal|anoLancamento|tempoMinutos|              genero|notaMedia|numeroVotos|generoArtista|      personagem|         nomeArtista|anoNascimento|anoFalecimento|           profissao|titulosMaisConhecidos|
+---------+--------------------+--------------------+-------------+------------+--------------------+---------+-----------+-------------+----------------+--------------------+-------------+--------------+--------------------+---------------------+
|tt0113074|Fist of the North...|Fist of the North...|         1995|         103|Action,Fantasy,Sc...|      3.9|       3417|        actor|        Kenshirô|        Gary Daniels|         1963|          null|actor,producer,st...| tt6156656,tt13202...|
|tt01130

Número de linhas antes IMDB:  1045161


Número de linhas depois IMDB:  166
Número de linhas TMDB:  49


## 3. 
Filtro as Series do BD Local com lista de series do TMDB

In [43]:
# Converter a coluna 'name' e a substring de 'first_air_date' do DataFrame JSON para o DataFrame do TMDB
df_series_nomes_datas = df_series_TMDB.select(col('name'), substring(col('first_air_date'), 1, 4).alias('dataLancamento'))

# Realizar a filtragem dos títulos e anos de lançamento
df_IMDB_filtrado_series = df_series_IMDB.join(df_series_nomes_datas,
                                         (df_series_IMDB['tituloPincipal'].contains(df_series_nomes_datas['name'])) &
                                         (df_series_IMDB['anoLancamento'] == df_series_nomes_datas['dataLancamento']),
                                         'inner')

# Remover as colunas 'name' e 'dataLancamento' do DataFrame
df_IMDB_filtrado_series = df_IMDB_filtrado_series.drop('name', 'dataLancamento')

df_IMDB_filtrado_series = df_IMDB_filtrado_series.distinct()

# Renomear a coluna 'tituloPincipal' para 'tituloPrincipal' no DataFrame df_IMDB_filtrado_filmes
df_IMDB_filtrado_series = df_IMDB_filtrado_series.withColumnRenamed("tituloPincipal", "tituloPrincipal")

# Exibir resultado
df_IMDB_filtrado_series.show(10)
print("Número de linhas antes IMDB: ", df_series_IMDB.count())
print("Número de linhas IMDB: ", df_IMDB_filtrado_series.count())
print("Número de linhas TMDB: ", df_series_TMDB.count())

+---------+--------------------+--------------------+-------------+----------+------------+--------------------+---------+-----------+-------------+--------------------+--------------+-------------+--------------+--------------------+---------------------+
|       id|     tituloPrincipal|      tituloOriginal|anoLancamento|anoTermino|tempoMinutos|              genero|notaMedia|numeroVotos|generoArtista|          personagem|   nomeArtista|anoNascimento|anoFalecimento|           profissao|titulosMaisConhecidos|
+---------+--------------------+--------------------+-------------+----------+------------+--------------------+---------+-----------+-------------+--------------------+--------------+-------------+--------------+--------------------+---------------------+
|tt0062582|         Mighty Jack|       Maitei Jyakku|         1968|      null|        null|       Action,Sci-Fi|      4.3|         34|        actor|Van Manuel Perez ...|  Fred Bossart|         null|          null|               a

Número de linhas IMDB:  22143
Número de linhas TMDB:  10000


## 4. 
Filtro as Series do BD Local que foram filtradas no passo 3

com a dos filmes do BD local que foram filtrados no passo 2

In [59]:
# Filtrar apenas os títulos das séries correspondentes aos filmes
df_series_correspondentes = df_IMDB_Filmes_resultado.select(col('tituloPrincipal').alias('titulo'))

# Realizar a junção entre os dataframes de séries e títulos correspondentes usando contains
df_IMDB_Series_resultado = df_IMDB_filtrado_series.join(df_series_correspondentes,
                                            df_IMDB_filtrado_series['tituloPrincipal'].contains(df_series_correspondentes['titulo']),
                                            'inner')

df_IMDB_Series_resultado = df_IMDB_Series_resultado.drop('titulo')

# Remover linhas duplicadas no resultado
df_IMDB_Series_resultado = df_IMDB_Series_resultado.distinct()

# Trata os valores "NA" na coluna "anoFalecimento"
df_IMDB_filtrado_series = df_IMDB_filtrado_series.withColumn("anoNascimento", when(col("anoNascimento") == "\\N", None).otherwise(col("anoNascimento")))
df_IMDB_filtrado_series = df_IMDB_filtrado_series.withColumn("anoFalecimento", when(col("anoFalecimento") == "\\N", None).otherwise(col("anoFalecimento")))
df_IMDB_filtrado_series = df_IMDB_filtrado_series.withColumn("anoTermino", when(col("anoTermino") == "\\N", None).otherwise(col("anoTermino")))
df_IMDB_filtrado_series = df_IMDB_filtrado_series.withColumn("tempoMinutos", when(col("tempoMinutos") == "\\N", None).otherwise(col("tempoMinutos")))

# Exibir resultado das séries correspondentes aos filmes
df_IMDB_Series_resultado.show(10)

print("Número de linhas IMDB antes: ", df_IMDB_filtrado_series.count())
print("Número de linhas no resultado: ", df_IMDB_Series_resultado.count())

+---------+--------------------+--------------------+-------------+----------+------------+--------------------+---------+-----------+-------------+--------------------+------------------+-------------+--------------+--------------------+---------------------+
|       id|     tituloPrincipal|      tituloOriginal|anoLancamento|anoTermino|tempoMinutos|              genero|notaMedia|numeroVotos|generoArtista|          personagem|       nomeArtista|anoNascimento|anoFalecimento|           profissao|titulosMaisConhecidos|
+---------+--------------------+--------------------+-------------+----------+------------+--------------------+---------+-----------+-------------+--------------------+------------------+-------------+--------------+--------------------+---------------------+
|tt1316554|        Black Butler|        Kuroshitsuji|         2008|      2011|          24|Action,Animation,...|      7.7|       8261|        actor|Drossel Keinz (2010)|Michael C. Pizzuto|         null|          null|

Número de linhas IMDB antes:  22143


Número de linhas no resultado:  171


## 5. 
Filtro as Series do TMDB com lista de Filmes do TMDB

In [55]:
# Filtrar apenas os títulos das séries correspondentes aos filmes
df_filmes_titulos = df_movies_TMDB.select(col('title').alias('titulo'))

# Realizar a junção entre os dataframes de séries e títulos correspondentes usando contains
df_TMDB_Series_resultado = df_series_TMDB.join(df_filmes_titulos,
                                            df_series_TMDB['name'].contains(df_filmes_titulos['titulo']),
                                            'inner')

df_TMDB_Series_resultado = df_TMDB_Series_resultado.drop('titulo','backdrop_path','poster_path')

# Remover linhas duplicadas no resultado
df_TMDB_Series_resultado = df_TMDB_Series_resultado.distinct()

# Trata os valores "NA" na coluna "anoFalecimento"
df_TMDB_Series_resultado = df_TMDB_Series_resultado.withColumn("first_air_date", when(col("first_air_date") == "", None).otherwise(col("first_air_date")))
df_TMDB_Series_resultado = df_TMDB_Series_resultado.withColumn("genre_ids", when(size(col("genre_ids")) == 0, None).otherwise(col("genre_ids")))
df_TMDB_Series_resultado = df_TMDB_Series_resultado.withColumn("overview", when(col("overview") == "", None).otherwise(col("overview")))

# Exibir resultado das séries correspondentes aos filmes
df_TMDB_Series_resultado.show(10)
print("Número de linhas antes TMDB: ", df_series_TMDB.count())
print("Número de linhas no resultado: ", df_TMDB_Series_resultado.count())

+--------------+--------------------+------+--------------------+--------------+-----------------+--------------------+--------------------+----------+------------+----------+
|first_air_date|           genre_ids|    id|                name|origin_country|original_language|       original_name|            overview|popularity|vote_average|vote_count|
+--------------+--------------------+------+--------------------+--------------+-----------------+--------------------+--------------------+----------+------------+----------+
|    2011-01-12|            [16, 35]| 55326|DD Fist of the No...|          [JP]|               ja|          DD北斗之拳|                null|     0.919|         6.0|         2|
|    2020-04-23|[16, 18, 10759, 1...| 90293|Ghost in the Shel...|          [JP]|               ja| 攻殻機動隊 SAC_2045|After a global fi...|    22.923|         6.6|        85|
|    1967-09-23|[10759, 16, 35, 1...| 30986|         Speed Racer|          [JP]|               ja|        マッハGoGoGo|Teenager Spee

## 6.
Elimina algumas colunas não relevantes do df_movies_TMDB 

In [68]:
df_TMDB_filmes_resultado = df_movies_TMDB.drop('adult','backdrop_path','poster_path','video')

df_movies_TMDB.show(5)
df_TMDB_filmes_resultado.show(5)

+-----+--------------------+-------------+------+-----------------+--------------------------------+--------------------+----------+--------------------+------------+--------------------+-----+------------+----------+
|adult|       backdrop_path|    genre_ids|    id|original_language|                  original_title|            overview|popularity|         poster_path|release_date|               title|video|vote_average|vote_count|
+-----+--------------------+-------------+------+-----------------+--------------------------------+--------------------+----------+--------------------+------------+--------------------+-----+------------+----------+
|false|/993utJnne28M1WNC...| [14, 28, 12]|455476|               en|            Knights of the Zo...|When a headstrong...|   304.591|/1nbFEdIFprRNSWcC...|  2023-04-27|Knights of the Zo...|false|         6.3|        49|
|false|/5hoS3nEkGGXUfmnu...| [28, 12, 14]|960704|               ja|  鋼の錬金術師 完結編 最後の錬成|The Elric brother...|     97.28|/AeyiuQUUs

## 6.

In [ ]:
"""
# Salvar df_IMDB_Filmes_resultado em formato Parquet
df_IMDB_Filmes_resultado.write.csv("/mnt/wsl/PHYSICALDRIVE2/Projects/Compasso/Azimute/Sprint_9/Desafio03_ETL/Tarefa_03/salvar_csv/df_IMDB_filtrado_filmes.csv")

# Salvar df_IMDB_Series_resultado em formato Parquet
df_IMDB_Series_resultado.write.csv("/mnt/wsl/PHYSICALDRIVE2/Projects/Compasso/Azimute/Sprint_9/Desafio03_ETL/Tarefa_03/salvar_csv/df_IMDB_filtrado_series.csv")

# Salvar df_TMDB_Series_resultado em formato Parquet
df_TMDB_Series_resultado.write.csv("/mnt/wsl/PHYSICALDRIVE2/Projects/Compasso/Azimute/Sprint_9/Desafio03_ETL/Tarefa_03/salvar_csv/df_IMDB_Series_resultado.scv")

# Salvar df_df_TMDB_filmes_resultado em formato Parquet
df_TMDB_filmes_resultado.write.csv("/mnt/wsl/PHYSICALDRIVE2/Projects/Compasso/Azimute/Sprint_9/Desafio03_ETL/Tarefa_03/salvar_csv/df_TMDB_Series_resultado.csv")
"""